# Introduction / Problem Statement

The other day I went to the comic book store to get a comic.  I live in St. Paul, Minnesota, so I went to a really nice comic book store near my house called, "The Source Comics".

Unfortunately, The Source did NOT have the comic book I wanted to pick up.  If I wanted to get it, I would have to drive 30 minutes to go to another comic book store and see if they had the comic book.  Although I like the Source a lot, it seems like there is quite often a very specific comic that I want, and this problem of mine keeps happening.

I wondered, is there a city in the US, where comic book stores are closer to each other?

# The Data

For my project I will repeatedly call the Foursquare API for the list of cities below and find which cities have the highest density of comic book stores.  I'm going to pretend for the sake of this project that the proximity of comic book stores is so vitally important to me, that I'm considering moving to avoid this VERY serious problem described above.  

I think I will repeatedly call the Foursquare API to identify comic book stores and their locations.  Then for each of the stores, I will call the google api to see which are within a 5 minute drive of each other.  The city with the most "within 5 minute" comic book stores will be the winner!  Then I can start packing.

I'm picking these cities because I don't want to go to a town that is too big...it should still be fairly similar to Minneapolis/St. Paul, because that is where I'm from:
1. Portland, Ore.
2. Milwaukee, WI
3. Columbus, OH
4. Denver
5. Kansas City
6. Madison, WI